<a href="https://colab.research.google.com/github/helpingstar/DL-study-179/blob/minyou/%EC%BA%90%EA%B8%80_Animal10%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import sys
from google.colab import drive
import os
import pandas as pd
import numpy as np
import cv2
import torch
from torch import nn
import torch.utils.data as data 
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import datasets #sample label을 저장 
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor 
from sklearn.model_selection import train_test_split



#splitfolders.ratio('C:\Users\TDA\Desktop\archive\raw-img',output="C: \Users\TDA\Desktop\archive", seed = 77, ratio =(0.8,0.1,0.1))
batch_size = 64






In [ ]:
class ImageTransform(): #이미지 전처리 

  def __init__(self, mean, std):
    self.data_transfrom = transforms.Compose([
          transforms.ToTensor(),
          transforms.Normalize(mean,std)
    ])

  def __call__(self, img):
    return self.data_transform(img)

In [ ]:


class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()

    self.layer = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5),
        nn.ReLu(),
        nn.Conv2d(in_channesl=16, out_channels=32, kernel_size=5),
        nn.ReLu(),
        
        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Conv2d(in_channels=32, out_Channels=64, kernel_size=5),
        nn.ReLu(),

        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.fc_layer = nn.Sequential(
        nn.Linear(64*3*3, 100),
        nn.ReLu(),
        nn.Linear(100,10)
    )
    def forwaerd(self, x):
      out = self.layer(x)
      out = out.view(50, -1)
      out = self.fc_layer(out)
      return out

In [41]:
from torchvision import models

#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu"

transfer_model = models.resnet50(pretrained = True)
#model = CNN()
transfer_model.fc = nn.Sequential(nn.Linear(transfer_model.fc.in_features, 500), nn.ReLU(),nn.Dropout(),nn.Linear(500,10))


#if torch.cuda.is_available():
    #device = torch.device("cuda")
#else:
    #device = torch.device("cpu")



In [42]:
transform= transforms.Compose([transforms.RandomRotation(degrees=20),
                        transforms.RandomHorizontalFlip(p=0.5),
                        transforms.Resize((64, 64)), # (h, w) 순서
                        transforms.ToTensor()])

root_dir = '/content/drive/MyDrive/archive/raw-img' # 이미지의 최상위 디렉토리
data  =ImageFolder(root_dir, transform=transform, target_transform=None)
data_size = len(data)




In [ ]:
train_dataset, test_dataset = train_test_split(data,test_size=0.2)
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True,num_workers=2)
test_dataloader = DataLoader(test_dataset, batch_size = batch_size, shuffle = True, num_workers=2)

In [ ]:
model = transfer_model
#model.to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(transfer_model.parameters(), lr=0.003)
epochs=1
loss_lst = []

for epoch in range(epochs):
  
  training_loss = 0.0
  valid_loss = 0.0
  model.train()
  for batch in train_dataloader:
    optimizer.zero_grad()
    inputs, target = batch
    #inputs = inputs.to(device)
    #target = target.to(device)
    output = model(inputs)
    print(output)

    loss = loss_func(output,target)
    loss.backward()
    optimizer.step()
    training_loss += loss.data.item()

  loss_lst.append(training_loss)

  model.eval()
  num_correct = 0
  num_examples = 0
  for batch in test_dataloader:
          inputs, targets = batch
          #inputs = inputs.to(device)
          output = model(inputs)
          #targets = targets.to(device)
          loss = loss_func(output,targets)
          valid_loss += loss.data.item()
          correct = torch.eq(torch.max(F.softmax(output), dim=1)[1],
                          targets).view(-1)
          num_correct += torch.sum(correct).item()
          num_examples += correct.shape[0]
  loss_val_lst.append(valid_loss)

  print('Epoch: {}, Training Loss: {:.2f}, \
  Validation Loss: {:.2f}, \
  accuracy = {:.2f}'.format(epoch, training_loss, \
  valid_loss, num_correct / num_examples))

tensor([[ 5.3982e-01,  3.8923e-01,  2.0161e-02, -5.3063e-01,  2.9687e-01,
         -5.9732e-01,  3.1846e-01,  1.9276e-01, -6.2086e-01, -2.7390e-01],
        [ 2.1487e-01, -3.1425e+00, -5.3297e+00,  8.9155e-01, -5.5179e-01,
          3.4066e-01, -5.0615e+00, -2.1851e+00,  3.4251e+00, -4.5478e-01],
        [ 3.2036e-01, -1.5788e-02, -3.1438e-01, -2.0692e-01,  1.1929e-01,
         -1.5522e-01, -2.9976e-01, -2.2255e-01,  1.3514e-01, -7.9504e-02],
        [-9.1558e-02, -1.1007e+00, -2.0813e+00,  8.3742e-01,  3.4518e-02,
         -6.8018e-01, -8.3873e-01, -1.0112e-01,  1.4721e+00, -8.4251e-02],
        [ 8.7908e-01,  3.3610e-02, -3.0672e-01, -8.7770e-01,  2.5465e-01,
         -5.3245e-02, -5.2526e-01, -3.7368e-01, -8.8412e-02, -2.3337e-01],
        [ 7.8856e-02, -7.6492e-01, -1.8473e+00,  8.8505e-01, -2.1154e-01,
         -1.8777e+00, -1.7048e-02,  2.8599e-01,  1.0130e+00, -6.0815e-02],
        [ 7.6139e-01,  5.7113e-01,  1.5571e-01, -9.0701e-01,  3.7530e-01,
         -7.6396e-01,  5.5511e-0